# DRUGS IN EUROPE
Drugs are a heavily debated issue. If we want people to take fewer drugs, what is the best way to do it? This project is about diving into the data to understand: Why are some countries outperforming others when it comes to making people not take drugs?

# 1 Motivation
<!-- What is your dataset?
Why did you choose this/these particular dataset(s)?
What was your goal for the end user's experience?
 -->
 

## 1.1 The datasets
For this project, we wanted to work on multiple different datasets and see what connections we could find between them. 

The majority of the datasets are from European Monitoring Centre for Drugs and Drug Addiction, while other "general" data (e.g.  population growth) came from other sources.

An overview of the relevant datasets can be found below:


| File                    	| Source                                                                                                      	| Row  	| Missing rows 	|
|-------------------------	|-------------------------------------------------------------------------------------------------------------	|------	|--------------	|
| cocaine-price.xlsx      	| https://www.emcdda.europa.eu/data/stats2020/ppp                                                             	| 663  	| 323          	|
| heroin-price.xlsx       	| https://www.emcdda.europa.eu/data/stats2020/ppp                                                             	| 663  	| 348          	|
| meth-price.xlsx         	| https://www.emcdda.europa.eu/data/stats2020/ppp                                                             	| 663  	| 565          	|
| Amphetamine-price.xlsx  	| https://www.emcdda.europa.eu/data/stats2020/ppp                                                             	| 663  	| 367          	|
| male-od.xlsx            	| https://www.emcdda.europa.eu/data/stats2020/drd                                                             	| 720  	| 120          	|
| female-od.xlsx          	| https://www.emcdda.europa.eu/data/stats2020/drd                                                             	| 720  	| 134          	|
| total-od.xlsx           	| https://www.emcdda.europa.eu/data/stats2020/drd                                                             	| 1020 	| 253          	|
| cocaine-purity.xlsx     	| https://www.emcdda.europa.eu/data/stats2020/ppp                                                             	| 544  	| 121          	|
| heroin-purity.xlsx      	| https://www.emcdda.europa.eu/data/stats2020/ppp                                                             	| 561  	| 292          	|
| meth-purity.xlsx        	| https://www.emcdda.europa.eu/data/stats2020/ppp                                                             	| 544  	| 285          	|
| Amphetamine-purity.xlsx 	| https://www.emcdda.europa.eu/data/stats2020/ppp                                                             	| 544  	| 174          	|
| population_2.xlsx       	| https://data.worldbank.org/indicator/SP.POP.TOTL                                                            	| 5016 	| 27           	|
| HIV-illness.xlsx        	| https://www.emcdda.europa.eu/data/stats2020/drid_en                                                         	| 330  	| 30           	|
| AIDS-illness.xlsx       	| https://www.emcdda.europa.eu/data/stats2020/drid_en                                                         	| 330  	| 40           	|
| drug_offences.csv       	| https://data.europa.eu/data/datasets/drug-law-offences-in-europe-emcdda-2020-statistical-bulletin?locale=en 	| 360  	| 37           	|
| **TOTAL**          | 	     | 13,341  	     | 3,116           	|

## 1.2 Why
The datasets were initially chosen because they could provide some insight into the market trends for illegal drug trafficking which was interesting as the initial idea was to use the data for recommending the location and type of drug to start trafficking. 

As funny as the joke was, it was decided to change focus to something less illegal with greater benefits for society.

## 1.3 End goal
The end goal then became to showcase societal consequences of drug trends of EU and European countries might learn from each other to combat the rising trend. 

Another end goal was to work on making a persuasive visualization, capable of showing the difference in handling of drugs across countries.

# 2 Basic stats
<!-- Write about your choices in data cleaning and preprocessing
Write a short section that discusses the dataset stats, containing key points/plots from your exploratory data analysis. -->

## 2.1 Data Cleaning
The data was often in a trends-format and needed a lot of cleaning to be useful. We utilized the following methods:
1. Re-organizing the dataset
2. Removing NaN
3. Renaming data to ensure merge

### 2.1.1. Re-organizing the dataset
An example is the drug number of drug offences:

In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import re
import plotly.io as pio
from statistics import *

# naming a layout theme for future reference
pio.templates["dtu"] = go.layout.Template(
    layout_colorway=['#990000', '#2F3EEA', '#1FD082', '#030F4F', '#F6D04D', '#FC7634', '#F7BBB1', '#DADADA', '#E83F48', '#79238E']
)

# setting Google color palette as default
pio.templates.default = "dtu"

df = pd.read_csv("drug_offences.csv", sep=";")
df = df.set_index("Country")

In order to use the data properly, the data needed to be transformed from a grid-format

In [2]:
new_df = []

for col, data in df.items():
    if not re.search('[a-zA-Z]', col) == None:
        continue
    new_df.append([data.index.to_list(), data.values.tolist(), [data.name]*len(df)])

liist = [list(zip(x,y,z)) for x, y, z in new_df]
newlist = [item for items in liist for item in items]
large_df = pd.DataFrame(newlist, columns=["Country", "offences", "Year"])
large_df.head(6)

,Country,offences,Year
0,Austria,47880.0,2018
1,Belgium,54749.0,2018
2,Bulgaria,2382.0,2018
3,Croatia,11179.0,2018
4,Cyprus,1168.0,2018
5,Czechia,NaN,2018


### 2.1.2. Removing NaN's
At this point, many NaN's were still present. Therefore, we chose to remove the rows containing NaN's.

In [3]:
large_df = large_df.dropna()
large_df.head(5)

,Country,offences,Year
0,Austria,47880.0,2018
1,Belgium,54749.0,2018
2,Bulgaria,2382.0,2018
3,Croatia,11179.0,2018
4,Cyprus,1168.0,2018


Another key step was ensuring that the different datasets could be merged properly. This was ensured through casting the columns as specific data types as well as specifying which columns to merge upon.
### 2.1.3. Renaming Values
In some datasets, countries had different names (e.g. "czechia" vs "Czech Republic"). In those cases, we renamed the values to ensure that the same country had only one name across datasets.

In [16]:
rename = lambda x : x.replace("-A", "").replace("-B", "").replace("Czech Republic", "Czechia").replace(" *", "") if type(x) == str else x

## 2.2 Dataset stats

For each of the datasets shown above, a quick overview of the basic stats can be found below. We are analyzing the following data:
* Prices
* Purity
* Overdoses

First of all, we need to define a helper function, add_rows that combines multiple files of grid-format to row-format.

In [17]:
def add_rows(df_path, drug_name="", return_df=[], column_name="Price", category_name="Drug"):
    new_df = []
    
    if "xlsx" in df_path:
        df = pd.read_excel(df_path, skiprows=0)
    elif "csv" in df_path:
        df = pd.read_csv(df_path, skiprows=3)
    else:
        raise FileNotFoundError("Not csv or xlsx")
    
    df = df.set_index("Country")
    
    for col, data in df.items():
        if not re.search('[a-zA-Z]', str(col)) == None:
            #print(col)
            continue
        if category_name == "":
            new_df.append([data.index.to_list(), data.values.tolist(), [data.name]*len(df)])
        else:
            new_df.append([data.index.to_list(), data.values.tolist(), [data.name]*len(df), [drug_name]*len(df)])
    
    
    #print(new_df)
    if category_name == "":
        liist = [list(zip(x,y,z)) for x, y, z in new_df]
        newlist = [item for items in liist for item in items]
        large_df = pd.DataFrame(newlist, columns=["Country", column_name, "Year"])
    else:
        liist = [list(zip(x,y,z, o)) for x, y, z, o in new_df]
        newlist = [item for items in liist for item in items]
        large_df = pd.DataFrame(newlist, columns=["Country", column_name, "Year", category_name])
    
    large_df.Country = large_df.Country.apply(rename)
    return pd.concat([return_df, large_df])

def layout_figure(fig, title, xaxis, yaxis, legend):
    fig.update_layout(
        title="<b>" + title + "</b>",
        xaxis_title=xaxis,
        yaxis_title=yaxis,
        legend_title="Countries",
        font=dict(
            family="Segoe UI",
            size=14,
            color="Black"
        ),
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False),
        plot_bgcolor='white',
        title_x=0.5
    )
    return fig

### 2.2.1 Preparing datasets
Then, we prepare the datasets by transforming from grid to row.

In [18]:
drug_df = pd.DataFrame({'Year' : [], "Country":[], "Price":[], "Drug":[]})
for file_name, name in zip(["cocaine-price.xlsx", "heroin-price.xlsx", "meth-price.xlsx", "Amphetamine-price.xlsx"],
                ["cocaine", "heroin", "meth", "amphetamine"]):
    drug_df = add_rows(df_path=file_name, drug_name=name, return_df=drug_df, column_name="Price")

drug_df.Year = drug_df.Year.astype(int)
drug_df = drug_df.dropna()
drug_df.head()

,Year,Country,Price,Drug
1,2018,Belgium,56.01,cocaine
4,2018,Bulgaria,61.90,cocaine
5,2018,Croatia,65.00,cocaine
7,2018,Czechia,76.70,cocaine
8,2018,Denmark,91.00,cocaine


In [19]:
purity_df = pd.DataFrame({'Year' : [], "Country":[], "Purity":[], "Drug":[]})
for file_name, name in zip(["cocaine-purity.xlsx", "heroin-purity.xlsx", "meth-purity.xlsx", "Amphetamine-purity.xlsx"],
                ["cocaine", "heroin", "meth", "amphetamine"]):
    purity_df = add_rows(df_path=file_name, drug_name=name, return_df=purity_df, column_name="Purity")

purity_df.Year = purity_df.Year.astype(int)
purity_df = purity_df.dropna()
purity_df.head(3)

,Year,Country,Purity,Drug
0,2018,Austria,51.37,cocaine
1,2018,Belgium,72.25,cocaine
2,2018,Bulgaria,51.40,cocaine


In [20]:
od_df = pd.DataFrame({'Year' : [], "Country":[], "Overdose":[], "Drug":[]})
for file_name, name in zip(["male-od.xlsx", "female-od.xlsx", "total-od.xlsx"],
                ["male", "female", "total"]):
    od_df = add_rows(df_path=file_name, drug_name=name, return_df=od_df, column_name="Overdose")
    
od_df.head(3)

,Year,Country,Overdose,Drug
0,2018.0,Austria,141.0,male
1,2018.0,Belgium,NaN,male
2,2018.0,Bulgaria,18.0,male


In [21]:
pop_df = add_rows(df_path="population_2.xlsx", drug_name="", return_df=pd.DataFrame({'Year' : [], "Country":[], "Population":[]}), column_name="Population", category_name="").dropna()
pop_df.Year = pop_df.Year.astype(int)
pop_df.Population = pop_df.Population.astype(int)
pop_df.head(3)

,Year,Country,Population
0,2001,Aruba,92898
1,2001,Afghanistan,21606988
2,2001,Angola,16945753


### 2.2.2 Drug Prices
There is a large variance in pricing, with heroin being sold for everything between 15€ and 190€. On the other hand, meth is usually a lot cheaper.

Most of the drug prices are centered around a specific value with a few changes. Especially with meth, there's a huge discrepancy between the mean and the median, indicating a skewed distribution. 

In [22]:
histogram = go.Figure()
years = sorted(list(set(drug_df.Year)), reverse=False)
drugs = list(set(drug_df.Drug))

for drug in drugs:
    price = drug_df[(drug_df.Drug==drug)].Price
    histogram.add_trace(
        go.Histogram(x=price, name=drug, opacity=0.7, xbins=dict(size=10))
    )
    print("{:<15s} Min: {:<15.2f} \tMax: {:<15.2f}\t Mean: {:<15.2f}\t Median: {:<15.2f}".format(drug, min(price), max(price), mean(price), median(price)))
    
_ = histogram.update_layout(barmode='overlay')

cocaine         Min: 0.00            	Max: 175.00         	 Mean: 67.33          	 Median: 63.97          
heroin          Min: 0.00            	Max: 190.00         	 Mean: 56.15          	 Median: 45.00          
amphetamine     Min: 0.00            	Max: 89.80          	 Mean: 17.27          	 Median: 13.10          
meth            Min: 0.00            	Max: 172.00         	 Mean: 40.87          	 Median: 27.40          


For all four drugs, there are data points indicating a price of zero. This is assumed to be a data error, as giving drugs away for free seems odd. When the zeros are removed, the data is distributed as follows:

In [23]:
drug_df = drug_df[drug_df["Price"]>0]
for drug in drugs:
    price = drug_df[(drug_df.Drug==drug)].Price
    print("{:<15s} Min: {:<15.2f} \tMax: {:<15.2f}\t Mean: {:<15.2f}\t Median: {:<15.2f}".format(drug, min(price), max(price), mean(price), median(price)))
    
histogram

cocaine         Min: 36.40           	Max: 175.00         	 Mean: 67.93          	 Median: 64.00          
heroin          Min: 8.00            	Max: 190.00         	 Mean: 56.33          	 Median: 45.12          
amphetamine     Min: 3.00            	Max: 89.80          	 Mean: 17.39          	 Median: 13.13          
meth            Min: 5.00            	Max: 172.00         	 Mean: 41.29          	 Median: 27.80          


### 2.2.3 Drug Purity
The purity of drugs are closer lumped together. Almost no drugs are above 80% pure.

In [24]:
histogram_2 = go.Figure()

years = sorted(list(set(purity_df.Year)), reverse=False)
drugs = list(set(purity_df.Drug))

for drug in drugs:
    purity = purity_df[(purity_df.Drug==drug)].Purity
    histogram_2.add_trace(
        go.Histogram(x=purity, name=drug, opacity=0.7, xbins=dict(size=10))
    )
    print("{:<15s} Min: {:<15.2f} \tMax: {:<15.2f}\t Mean: {:<15.2f}\t Median: {:<15.2f}".format(drug, min(purity), max(purity), mean(purity), median(purity)))
    
_ = histogram_2.update_layout(barmode='overlay')

cocaine         Min: 11.80           	Max: 90.34          	 Mean: 45.05          	 Median: 43.90          
heroin          Min: 0.00            	Max: 57.00          	 Mean: 22.80          	 Median: 20.00          
amphetamine     Min: 0.60            	Max: 62.10          	 Mean: 19.86          	 Median: 16.20          
meth            Min: 5.00            	Max: 90.40          	 Mean: 45.05          	 Median: 44.00          


Furthermore, heroin has a minimum purity of 0. This is again assumed to be a data error, as "heroin" would have to contain some heroin to be classified as heroin. Because would it really be heroin, then?

Before we get too deeply into the philosophical nature of the question, we'll remove those points from the data.

In [25]:
purity_df = purity_df[purity_df["Purity"]>0]
for drug in drugs:
    purity = purity_df[(purity_df.Drug==drug)].Purity
    print("{:<15s} Min: {:<15.2f} \tMax: {:<15.2f}\t Mean: {:<15.2f}\t Median: {:<15.2f}".format(drug, min(purity), max(purity), mean(purity), median(purity)))
    
histogram_2

cocaine         Min: 11.80           	Max: 90.34          	 Mean: 45.05          	 Median: 43.90          
heroin          Min: 5.80            	Max: 57.00          	 Mean: 22.88          	 Median: 20.00          
amphetamine     Min: 0.60            	Max: 62.10          	 Mean: 19.86          	 Median: 16.20          
meth            Min: 5.00            	Max: 90.40          	 Mean: 45.05          	 Median: 44.00          


### 2.2.4 Overdoses
In order to categorize the overdoses, we split the countries into "eastern" and "western" Europe. Afterwards, we can calculate the overdoses per thousand inhabitants, giving us a quick overview of the total ODs within from 2004 and 2018. 

It is clear to see that a small list of countries account for a huge part of the total overdoses.

In [26]:
od_pm = od_df.loc[(od_df.Drug=="total") & (od_df.Year > 2004) & (2018 > od_df.Year)].merge(pop_df)
od_pm["od_pm"] = od_pm.Overdose / od_pm.Population * 1000

od_total = od_pm.groupby(["Country"], as_index=False).agg("sum")
od_total = pd.read_excel("eastern.xlsx").merge(od_total, on="Country").sort_values("od_pm", ascending=False)
west = od_total[od_total.West==True]
east = od_total[od_total.West==False]

od_fig_2 = go.Figure()
od_fig_2.add_trace(go.Scatter(x = od_total.Country, y = [od_total.od_pm.mean()]*len(od_total), name="EU Average", mode="lines"))
od_fig_2.add_trace(go.Bar(x = west.Country, y = west.od_pm, name="Western Europe"))
od_fig_2.add_trace(go.Bar(x = east.Country, y = east.od_pm, name="Eastern Europe"))
od_fig_2.update_layout(barmode='stack', xaxis={'categoryorder':'total descending'})

layout_figure(od_fig_2, "Western Europe has a lot of ODs", "Country", "Total overdoses per thousand from 2004 to 2018", "Categories")

# 3 Data analysis
<!-- Describe your data analysis and explain what you've learned about the dataset.
If relevant, talk about your machine-learning. -->
WHAT KIND OF DATA ANALYSIS ARE WE DOING?
## 3.1 What we've learned about the datasets
The main learning with this data is how much information is currently missing about European use of drugs.


## 3.2 Machine-learning
CHRISTIAN POULSEN

# 4 Genre
<!-- Which genre of data story did you use?
Which tools did you use from each of the 3 categories of Visual Narrative (Figure 7 in Segal and Heer). Why?
Which tools did you use from each of the 3 categories of Narrative Structure (Figure 7 in Segal and Heer). Why? -->

The data story primarily uses the Slide Show genre for narrating the story to the end user, as the visualizations and its accompanying text are contained in a clear sequence of narrative events.
Using the website for scrolling through the sequence of the story creates a consistent visual platform for helping the viewer orientate and track his/her progress through the visualizations.

## 4.1 Tools from Visual Narrative

## 4.2 Tools from Narrative Structure

WE COULD DO SOME HIGHLIGHTING: We've done that using "Look at Portugal"

LET'S CREATE THE MOTION BETWEEN EACH SECTION FOR ANIMATED TRANSISTIONS

Linear
Hovering, filtering
Captions and headlines, annotations etc

<!-- As the user steps through the presentation, the visualization main-tains aconsistent visual platform, changing only the content withineach panel while leaving the general layout of the visual elements in-tact. Each new slide alters the text in the left panel, while updating thechart in the right panel with animated transitions. A narrative is com-municated clearly through the interaction of the text in the left panelwith the annotations and graphic elements in the right panel, each en-riching the narrative throughmulti-messaging, providing related butdifferent  information  [20].   -->

# 5 Visualizations
<!-- 
Explain the visualizations you've chosen.
Why are they right for the story you want to tell? -->

Sometimes there is only a specific number that needs communicating. Instead of using graphs and charts, using the number directly puts more focus on the number and removes any clutter there else could be if other kinds of visualizations was used.

For plotting continous data, line graphs are used, as the line that connects the points are physically connected implying a connection.

## 5.1 Explanation of visualizations
Our visualizations were mostly guided towards non-technical people who are not used to seeing complex data visualisation.


## 5.2 Why are they right?


# 6 Discussion
<!-- Think critically about your creation

What went well?,
What is still missing? What could be improved?, Why? -->

## 6.1 What went well
We were able to create a simple and straight-forward website that quickly took the reader through a number of graphs. It let's the reader dive deeper into the data, as well as giving a high-level overview of the data. 

## 6.2 What is missing
As mentioned above, we didn't see a fit for Machine Learning, given our small amount of countries. The chance of overfitting would simply be too big. We would have loved to find a way of including machine learning.

Given the genre of our communication, it didn't seem fitting to use some of the more complex/unusual graphs that we've learned during the course. If we had to do the project again, we would add another section "for data experts" to showcase some of the other findings we've made.

# 7 Contributions
<!-- Who did what?

You should write (just briefly) which group member was the main responsible for which elements of the assignment. (I want you guys to understand every part of the assignment, but usually there is someone who took lead role on certain portions of the work. That's what you should explain). -->
* Christian Bøgelund was responsible for performing the data analysis as well as the visualizations and gaining an overview of the data at hand
* Christian Poulsen was mainly responsible for understanding the storyline and graphs needed to support that storyline
* Beta Ntakiyimana was responsible for setting up the website and embedding the graphs 